In [1]:
import re
import csv
from urllib.parse import urlparse, parse_qs

def parse_file(file_in, file_out):
    with open(file_in, 'r', encoding='utf-8') as fin, open(file_out, 'w', newline='', encoding='utf-8') as fout:
        csv_writer = csv.writer(fout)
        # Write the header
        csv_writer.writerow([
            'Method', 'Full URL', 'URL Path', 'Query Params', 'Content-Length', 'Content-Type',
            'User-Agent', 'Host', 'Cookie', 'Accept', 'Accept-Encoding', 'Accept-Charset',
            'Accept-Language', 'Pragma', 'Cache-Control', 'Connection', 'Body', 'Timestamp',
            'IP Address', 'Status Code'
        ])
        
        lines = fin.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.startswith(('GET', 'POST', 'PUT', 'DELETE')):
                # Extract Method
                method = line.split(' ')[0]
                # Extract Full URL
                full_url = line.split(' ')[1]
                # Extract URL Path
                url_path = urlparse(full_url).path
                # Extract Query Parameters (if any)
                query_params = parse_qs(urlparse(full_url).query)
                query_params_str = '&'.join([f'{k}={v[0]}' for k, v in query_params.items()])
                
                # Initialize other fields
                content_length = ''
                content_type = ''
                user_agent = ''
                host = ''
                cookie = ''
                accept = ''
                accept_encoding = ''
                accept_charset = ''
                accept_language = ''
                pragma = ''
                cache_control = ''
                connection = ''
                body = ''
                timestamp = ''
                ip_address = ''  # Assuming IP address is not present in the line
                status_code = ''  # Assuming status code is not present in the line
                
                # Loop through headers and body
                j = 1
                while i + j < len(lines) and not lines[i + j].strip() == '':
                    header_line = lines[i + j].strip()
                    if header_line.startswith('Content-Length:'):
                        content_length = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Content-Type:'):
                        content_type = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('User-Agent:'):
                        user_agent = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Host:'):
                        host = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cookie:'):
                        cookie = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept:'):
                        accept = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Encoding:'):
                        accept_encoding = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Charset:'):
                        accept_charset = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Language:'):
                        accept_language = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Pragma:'):
                        pragma = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cache-Control:'):
                        cache_control = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Connection:'):
                        connection = header_line.split(':', 1)[1].strip()
                    j += 1
                
                # The body is the line after the headers
                if i + j < len(lines):
                    body = lines[i + j].strip()
                
                # Append the extracted features
                csv_writer.writerow([
                    method, full_url, url_path, query_params_str, content_length, content_type,
                    user_agent, host, cookie, accept, accept_encoding, accept_charset,
                    accept_language, pragma, cache_control, connection, body, timestamp,
                    ip_address, status_code
                ])
                
                i += j + 1
            else:
                i += 1



In [2]:

parse_file('anomalousTrafficTest.txt', 'anomalous_parsed_data.csv')

In [3]:
parse_file('normalTrafficTraining.txt', 'normal_parsed_data.csv')

In [4]:
import re
import csv
from urllib.parse import urlparse, parse_qs

def parse_file(file_in, file_out):
    with open(file_in, 'r', encoding='utf-8') as fin, open(file_out, 'w', newline='', encoding='utf-8') as fout:
        csv_writer = csv.writer(fout)
        # Write the header
        csv_writer.writerow([
            'Method', 'Full URL', 'URL Path', 'Query Params', 'Body Params', 'Content-Length', 
            'Content-Type', 'User-Agent', 'Host', 'Cookie', 'Accept', 'Accept-Encoding', 
            'Accept-Charset', 'Accept-Language', 'Pragma', 'Cache-Control', 'Connection'
        ])
        
        lines = fin.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.startswith(('GET', 'POST', 'PUT', 'DELETE')):
                # Extract Method
                method = line.split(' ')[0]
                # Extract Full URL
                full_url = line.split(' ')[1]
                # Extract URL Path
                url_path = urlparse(full_url).path
                # Extract Query Parameters (if any)
                query_params = parse_qs(urlparse(full_url).query)
                query_params_str = '; '.join([f'{k}={v[0]}' for k, v in query_params.items()])
                
                # Initialize other fields
                content_length = ''
                content_type = ''
                user_agent = ''
                host = ''
                cookie = ''
                accept = ''
                accept_encoding = ''
                accept_charset = ''
                accept_language = ''
                pragma = ''
                cache_control = ''
                connection = ''
                body_params = ''
                
                headers = {}
                body = ''
                
                # Loop through headers and body
                j = 1
                while i + j < len(lines) and not lines[i + j].strip() == '':
                    header_line = lines[i + j].strip()
                    if header_line.startswith('Content-Length:'):
                        content_length = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Content-Type:'):
                        content_type = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('User-Agent:'):
                        user_agent = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Host:'):
                        host = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cookie:'):
                        cookie = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept:'):
                        accept = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Encoding:'):
                        accept_encoding = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Charset:'):
                        accept_charset = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Language:'):
                        accept_language = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Pragma:'):
                        pragma = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cache-Control:'):
                        cache_control = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Connection:'):
                        connection = header_line.split(':', 1)[1].strip()
                    else:
                        # Collect all header fields
                        header_key, header_value = map(str.strip, header_line.split(':', 1))
                        headers[header_key] = header_value
                    
                    j += 1
                
                # The body is the line after the headers
                if i + j < len(lines):
                    body = lines[i + j].strip()
                    # Extract body parameters (if the body is URL-encoded)
                    body_params = parse_qs(body)
                    body_params_str = '; '.join([f'{k}={v[0]}' for k, v in body_params.items()])

                # Append the extracted features
                csv_writer.writerow([
                    method, full_url, url_path, query_params_str, body_params_str, content_length, 
                    content_type, user_agent, host, cookie, accept, accept_encoding, 
                    accept_charset, accept_language, pragma, cache_control, connection
                ])
                
                i += j + 1
            else:
                i += 1




In [5]:
parse_file('anomalousTrafficTest.txt', 'anomalous_parsed_data.csv')


In [6]:
parse_file('normalTrafficTraining.txt', 'normal_parsed_data.csv')

removed features such as value of cookie as it is specific to the request

In [9]:
import re
import csv
from urllib.parse import urlparse, parse_qs
from datetime import datetime

def extract_parameters(param_str):
    """
    Extracts parameters from a URL-encoded string and returns as a formatted string.
    """
    params = parse_qs(param_str)
    return '; '.join([f'{k}={v[0]}' for k, v in params.items()])

def parse_file(file_in, file_out):
    with open(file_in, 'r', encoding='utf-8') as fin, open(file_out, 'w', newline='', encoding='utf-8') as fout:
        csv_writer = csv.writer(fout)
        # Write the header
        csv_writer.writerow([
            'Method', 'Full URL', 'URL Path', 'Query Params', 'Body Params', 
            'Content-Length', 'Content-Type', 'User-Agent', 'Host','Accept', 
            'Accept-Encoding', 'Accept-Charset', 'Accept-Language', 'Pragma', 'Cache-Control', 
            'Connection', 'Body Length', 'URL Length'
        ])
        
        lines = fin.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.startswith(('GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'TRACE', 'PATCH', 'OPTIONS')):
                # Example timestamp extraction (assuming it's available in the log)
                # This part needs to be adapted to your log format if a timestamp is present.
                timestamp = 'N/A'  # Placeholder for timestamp extraction
                
                # Extract Method
                method = line.split(' ')[0]
                
                # Extract Full URL
                full_url = line.split(' ')[1]
                
                # Extract URL Path
                url_path = urlparse(full_url).path
                
                # Extract Query Parameters
                query_params_str = extract_parameters(urlparse(full_url).query)
                
                # Initialize other fields
                content_length = ''
                content_type = ''
                user_agent = ''
                host = ''
                accept = ''
                accept_encoding = ''
                accept_charset = ''
                accept_language = ''
                pragma = ''
                cache_control = ''
                connection = ''
                body_params_str = ''
                
                headers = {}
                body = ''
                
                # Loop through headers and body
                j = 1
                while i + j < len(lines) and not lines[i + j].strip() == '':
                    header_line = lines[i + j].strip()
                    if header_line.startswith('Content-Length:'):
                        content_length = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Content-Type:'):
                        content_type = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('User-Agent:'):
                        user_agent = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Host:'):
                        host = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept:'):
                        accept = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Encoding:'):
                        accept_encoding = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Charset:'):
                        accept_charset = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Language:'):
                        accept_language = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Pragma:'):
                        pragma = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cache-Control:'):
                        cache_control = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Connection:'):
                        connection = header_line.split(':', 1)[1].strip()
                    j += 1
                
                # The body is the line after the headers
                if i + j < len(lines):
                    body = lines[i + j].strip()
                    # Extract body parameters (if the body is URL-encoded)
                    body_params_str = extract_parameters(body)
                
                # Calculate lengths for additional features
                body_length = len(body)
                url_length = len(full_url)
                    
                # Append the extracted features to the CSV
                csv_writer.writerow([
                    method, full_url, url_path, query_params_str, body_params_str, 
                    content_length, content_type, user_agent, host, accept, 
                    accept_encoding, accept_charset, accept_language, pragma, cache_control, 
                    connection, body_length, url_length
                ])
                
                i += j + 1
            else:
                i += 1




In [10]:
parse_file('anomalousTrafficTest.txt', 'anomalous_parsed_data.csv')


In [11]:
parse_file('normalTrafficTraining.txt', 'normal_parsed_data.csv')

In [54]:
import re
import csv
import xml.etree.ElementTree as ET
from urllib.parse import urlparse, parse_qs
from datetime import datetime

def extract_parameters(param_str):
    """
    Extracts parameters from a URL-encoded string and returns as a formatted string.
    """
    params = parse_qs(param_str)
    return '; '.join([f'{k}={v[0]}' for k, v in params.items()])

def has_special_characters(s):
    """
    Checks if the string contains special characters that could be used in SQL injection.
    """
    sql_injection_patterns = [
        r"'",       # Single quote
        r"--",      # SQL comment
        r"\*",      # Asterisk
        r"#",       # Hash (another comment style)
        r"\bSELECT\b",   # SQL SELECT statement
        r"\bUNION\b",    # SQL UNION keyword
        r"\bINSERT\b",   # SQL INSERT statement
        r"\bUPDATE\b",   # SQL UPDATE statement
        r"\bDELETE\b",   # SQL DELETE statement
        r"\bDROP\b",     # SQL DROP statement
        r"\bFROM\b",     # SQL FROM keyword
        r"\bWHERE\b"     # SQL WHERE keyword
    ]
    
    return any(re.search(pattern, s, re.IGNORECASE) for pattern in sql_injection_patterns)

def parse_txt_file(file_in):
    """
    Parses a .txt file and returns a list of log entries.
    """
    with open(file_in, 'r', encoding='utf-8') as fin:
        return fin.readlines()

def parse_xml_file(file_in):
    """
    Parses a .xml file and returns a list of log entries.
    """
    tree = ET.parse(file_in)
    root = tree.getroot()
    logs = []

    for log_entry in root.findall('.//log_entry'):
        log_lines = []
        for line in log_entry.findall('line'):
            log_lines.append(line.text.strip())
        logs.append('\n'.join(log_lines))
    
    return logs

def parse_file(file_in, file_out):
    # Determine file type
    if file_in.endswith('.txt'):
        lines = parse_txt_file(file_in)
    elif file_in.endswith('.xml'):
        lines = parse_xml_file(file_in)
    else:
        raise ValueError("Unsupported file type. Only .txt and .xml files are supported.")

    with open(file_out, 'w', newline='', encoding='utf-8') as fout:
        csv_writer = csv.writer(fout)
        # Write the header
        csv_writer.writerow([
            'Method', 'Full URL', 'URL Path', 'Query Params', 'Query Params Length', 'Number of Query Params', 'Body Params', 
            'Content-Length', 'Content-Type', 'User-Agent', 'Host', 'Accept', 
            'Accept-Encoding', 'Accept-Charset', 'Accept-Language', 'Pragma', 'Cache-Control', 
            'Connection', 'Body Length', 'URL Length', 'Special Characters in URL', 'Special Characters in Query Params'
        ])
        
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.startswith(('GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'TRACE', 'PATCH', 'OPTIONS')):
                # Example timestamp extraction (assuming it's available in the log)
                timestamp = 'N/A'  # Placeholder for timestamp extraction
                
                # Extract Method
                method = line.split(' ')[0]
                
                # Extract Full URL
                full_url = line.split(' ')[1]
                
                # Extract URL Path
                url_path = urlparse(full_url).path
                
                # Extract Query Parameters
                query_params = parse_qs(urlparse(full_url).query)
                query_params_str = extract_parameters(urlparse(full_url).query)
                query_params_length = len(urlparse(full_url).query)
                num_query_params = len(query_params)
                
                # Check for special characters
                special_chars_in_url = has_special_characters(url_path)
                special_chars_in_query_params = any(has_special_characters(v[0]) for v in query_params.values())
                
                # Initialize other fields
                content_length = ''
                content_type = ''
                user_agent = ''
                host = ''
                accept = ''
                accept_encoding = ''
                accept_charset = ''
                accept_language = ''
                pragma = ''
                cache_control = ''
                connection = ''
                body_params_str = ''
                
                headers = {}
                body = ''
                
                # Loop through headers and body
                j = 1
                while i + j < len(lines) and not lines[i + j].strip() == '':
                    header_line = lines[i + j].strip()
                    if header_line.startswith('Content-Length:'):
                        content_length = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Content-Type:'):
                        content_type = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('User-Agent:'):
                        user_agent = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Host:'):
                        host = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept:'):
                        accept = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Encoding:'):
                        accept_encoding = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Charset:'):
                        accept_charset = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Language:'):
                        accept_language = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Pragma:'):
                        pragma = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cache-Control:'):
                        cache_control = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Connection:'):
                        connection = header_line.split(':', 1)[1].strip()
                    j += 1
                
                # The body is the line after the headers
                if i + j < len(lines):
                    body = lines[i + j].strip()
                    # Extract body parameters (if the body is URL-encoded)
                    body_params_str = extract_parameters(body)
                
                # Calculate lengths for additional features
                body_length = len(body)
                url_length = len(full_url)
                    
                # Append the extracted features to the CSV
                csv_writer.writerow([
                    method, full_url, url_path, query_params_str, query_params_length, num_query_params, body_params_str, 
                    content_length, content_type, user_agent, host, accept, 
                    accept_encoding, accept_charset, accept_language, pragma, cache_control, 
                    connection, body_length, url_length, special_chars_in_url, special_chars_in_query_params
                ])
                
                i += j + 1
            else:
                i += 1


In [19]:
parse_file('anomalousTrafficTest.txt', 'anomalous_parsed_data2.csv')


In [20]:
anomaly_file_parsed = 'anomalous_parsed_data2.csv'

In [21]:
import pandas as pd

In [22]:
anomaly_file_parsed=pd.read_csv('anomalous_parsed_data2.csv')

In [23]:
anomaly_file_parsed.head()

,Method,Full URL,URL Path,Query Params,Query Params Length,Number of Query Params,Body Params,Content-Length,Content-Type,User-Agent,...,Accept-Encoding,Accept-Charset,Accept-Language,Pragma,Cache-Control,Connection,Body Length,URL Length,Special Characters in URL,Special Characters in Query Params
0,GET,http://localhost:8080/tienda1/publico/anadir.j...,/tienda1/publico/anadir.jsp,id=2; nombre=Jam�n Ib�rico; precio=85; cantida...,146,5,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,195,False,True
1,POST,http://localhost:8080/tienda1/publico/anadir.jsp,/tienda1/publico/anadir.jsp,NaN,0,0,NaN,146.0,application/x-www-form-urlencoded,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,48,False,False
2,GET,http://localhost:8080/tienda1/publico/anadir.j...,/tienda1/publico/anadir.jsp,id=2/; nombre=Jam�n Ib�rico; precio=85; cantid...,77,5,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,126,False,False
3,POST,http://localhost:8080/tienda1/publico/anadir.jsp,/tienda1/publico/anadir.jsp,NaN,0,0,NaN,77.0,application/x-www-form-urlencoded,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,48,False,False
4,GET,http://localhost:8080/asf-logo-wide.gif~,/asf-logo-wide.gif~,NaN,0,0,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,40,False,False


In [24]:
import numpy as np 


In [25]:
n_features=anomaly_file_parsed.shape[1]
n_samples =anomaly_file_parsed.shape[0]

In [26]:
print(f'number of features: {n_features}')
missing_values_count = anomaly_file_parsed.isnull().sum()
missing_values_count[0:n_features]

number of features: 22


Method                                    0
Full URL                                  0
URL Path                                214
Query Params                          15468
Query Params Length                       0
Number of Query Params                    0
Body Params                           25065
Content-Length                        15088
Content-Type                          15088
User-Agent                                0
Host                                      0
Accept                                    0
Accept-Encoding                           0
Accept-Charset                            0
Accept-Language                           0
Pragma                                    0
Cache-Control                         25065
Connection                                0
Body Length                               0
URL Length                                0
Special Characters in URL                 0
Special Characters in Query Params        0
dtype: int64

In [27]:
total_cells = np.prod(anomaly_file_parsed.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing/total_cells) * 100
print('percentage missing:',(f'{percent_missing:.2f}') ,'%')

percentage missing: 17.41 %


In [28]:
for feature in anomaly_file_parsed.columns:
    if feature in anomaly_file_parsed.columns:
        unique_count = anomaly_file_parsed[feature].nunique()
        print(f"Number of unique values for {feature}: {unique_count}")
    else:
        print(f"Column '{feature}' does not exist in the DataFrame.")

Number of unique values for Method: 3
Number of unique values for Full URL: 8666
Number of unique values for URL Path: 1612
Number of unique values for Query Params: 7042
Number of unique values for Query Params Length: 383
Number of unique values for Number of Query Params: 5
Number of unique values for Body Params: 0
Number of unique values for Content-Length: 382
Number of unique values for Content-Type: 1
Number of unique values for User-Agent: 1
Number of unique values for Host: 2
Number of unique values for Accept: 1
Number of unique values for Accept-Encoding: 1
Number of unique values for Accept-Charset: 1
Number of unique values for Accept-Language: 1
Number of unique values for Pragma: 1
Number of unique values for Cache-Control: 0
Number of unique values for Connection: 1
Number of unique values for Body Length: 1
Number of unique values for URL Length: 416
Number of unique values for Special Characters in URL: 2
Number of unique values for Special Characters in Query Params

In [34]:
anomaly_file_parsed.shape[0]

25065

In [35]:
parse_file('normalTrafficTraining.txt', 'normal_parsed_data2.csv')


In [36]:
normal_file_parsed = 'normal_parsed_data2.csv'

In [37]:
normal_file_parsed=pd.read_csv('normal_parsed_data2.csv')

In [38]:
normal_file_parsed.head()

,Method,Full URL,URL Path,Query Params,Query Params Length,Number of Query Params,Body Params,Content-Length,Content-Type,User-Agent,...,Accept-Encoding,Accept-Charset,Accept-Language,Pragma,Cache-Control,Connection,Body Length,URL Length,Special Characters in URL,Special Characters in Query Params
0,GET,http://localhost:8080/tienda1/index.jsp,/tienda1/index.jsp,NaN,0,0,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,39,False,False
1,GET,http://localhost:8080/tienda1/publico/anadir.j...,/tienda1/publico/anadir.jsp,id=3; nombre=Vino Rioja; precio=100; cantidad=...,68,5,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,117,False,False
2,POST,http://localhost:8080/tienda1/publico/anadir.jsp,/tienda1/publico/anadir.jsp,NaN,0,0,NaN,68.0,application/x-www-form-urlencoded,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,48,False,False
3,GET,http://localhost:8080/tienda1/publico/autentic...,/tienda1/publico/autenticar.jsp,modo=entrar; login=choong; pwd=d1se3ci�n; reme...,63,5,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,116,False,False
4,POST,http://localhost:8080/tienda1/publico/autentic...,/tienda1/publico/autenticar.jsp,NaN,0,0,NaN,63.0,application/x-www-form-urlencoded,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,NaN,close,0,52,False,False


In [39]:
n_features=normal_file_parsed.shape[1]
n_samples =normal_file_parsed.shape[0]

In [43]:
print(f'number of features: {n_features}')
missing_values_count = normal_file_parsed.isnull().sum()
missing_values_count[0:n_features]

number of features: 22


Method                                    0
Full URL                                  0
URL Path                                  0
Query Params                          28000
Query Params Length                       0
Number of Query Params                    0
Body Params                           36000
Content-Length                        28000
Content-Type                          28000
User-Agent                                0
Host                                      0
Accept                                    0
Accept-Encoding                           0
Accept-Charset                            0
Accept-Language                           0
Pragma                                    0
Cache-Control                         36000
Connection                                0
Body Length                               0
URL Length                                0
Special Characters in URL                 0
Special Characters in Query Params        0
dtype: int64

In [42]:
total_cells = np.prod(normal_file_parsed.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing/total_cells) * 100
print('percentage missing:',(f'{percent_missing:.2f}') ,'%')

percentage missing: 12.12 %


In [44]:
for feature in normal_file_parsed.columns:
    if feature in normal_file_parsed.columns:
        unique_count = normal_file_parsed[feature].nunique()
        print(f"Number of unique values for {feature}: {unique_count}")
    else:
        print(f"Column '{feature}' does not exist in the DataFrame.")

Number of unique values for Method: 2
Number of unique values for Full URL: 4840
Number of unique values for URL Path: 28
Number of unique values for Query Params: 4812
Number of unique values for Query Params Length: 117
Number of unique values for Number of Query Params: 5
Number of unique values for Body Params: 0
Number of unique values for Content-Length: 116
Number of unique values for Content-Type: 1
Number of unique values for User-Agent: 1
Number of unique values for Host: 1
Number of unique values for Accept: 1
Number of unique values for Accept-Encoding: 1
Number of unique values for Accept-Charset: 1
Number of unique values for Accept-Language: 1
Number of unique values for Pragma: 1
Number of unique values for Cache-Control: 0
Number of unique values for Connection: 1
Number of unique values for Body Length: 1
Number of unique values for URL Length: 124
Number of unique values for Special Characters in URL: 1
Number of unique values for Special Characters in Query Params: 

## Tokenization

In [10]:
import csv
import re
import pandas as pd

def tokenize_field(field_value):
    # Tokenize the field value by splitting on non-alphanumeric characters and whitespace
    tokens = re.split(r'\W+|_', field_value)
    return [token for token in tokens if token]  # Remove empty tokens

def tokenize_http_request_row(row):
    tokens = []
    for field in row:
        tokens.extend(tokenize_field(field))
    return tokens

def process_csv(input_file_path, output_file_path):
    all_tokens = []

    # Read the CSV file
    with open(input_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Skip the header row if present

        # Tokenize each row in the CSV file
        for row in reader:
            row_tokens = tokenize_http_request_row(row)
            all_tokens.append(row_tokens)

    # Write tokenized data to a new CSV file
    with open(output_file_path, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        for tokens in all_tokens:
            writer.writerow(tokens)

def main():
    input_file_path = 'anomalous_parsed_data2.csv'  # Replace with your CSV file path
    output_file_path = 'anomalous_tokenized_data2.csv'  # Path to save the tokenized data
    process_csv(input_file_path, output_file_path)
    print(f"Tokenized data has been saved to {output_file_path}")

if __name__ == '__main__':
    main()


Tokenized data has been saved to anomalous_tokenized_data2.csv


## Vectorization Using TFIDF

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def read_tokenized_csv(file_path):
    # Attempt to read the CSV file with error handling
    try:
        df = pd.read_csv(file_path, header=None, error_bad_lines=False, quoting=3)  # quoting=3 to handle any unexpected quoting
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None
    
    # Combine tokens from each row into a single string
    df_combined = df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    return df_combined

def perform_tfidf_transformation(text_data):
    # Initialize the TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    # Fit and transform the data
    tfidf_matrix = vectorizer.fit_transform(text_data)
    return tfidf_matrix, vectorizer.get_feature_names_out()

def save_tfidf_to_csv(tfidf_matrix, feature_names, output_file_path):
    # Convert the TF-IDF matrix to a DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
    # Save the DataFrame to a CSV file
    tfidf_df.to_csv(output_file_path, index=False)

def main():
    tokenized_file_path = 'anomalous_tokenized_data2.csv'  # Replace with your tokenized CSV file path
    tfidf_output_file_path = 'anomalous_vectorised_data2'  # Path to save the TF-IDF matrix

    # Step 1: Read the tokenized CSV file
    text_data = read_tokenized_csv(tokenized_file_path)
    if text_data is None:
        print("Failed to read tokenized CSV file.")
        return

    # Step 2: Perform TF-IDF transformation
    tfidf_matrix, feature_names = perform_tfidf_transformation(text_data)

    # Step 3: Save the TF-IDF matrix to a CSV file
    save_tfidf_to_csv(tfidf_matrix, feature_names, tfidf_output_file_path)

    print(f"TF-IDF matrix has been saved to {tfidf_output_file_path}")

if __name__ == '__main__':
    main()


Error reading CSV file: read_csv() got an unexpected keyword argument 'error_bad_lines'
Failed to read tokenized CSV file.


## Feature Selection using Information Gain Algorithm

In [14]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

# Load your dataset
data = pd.read_csv('All.csv')

# Identify the target column (assuming it's the last column)
target_column = data.columns[-1]
X = data.iloc[:, :-1]  # All columns except the last one
y = data[target_column]  # Last column as the target

# Encode categorical features and target variable if necessary
X_encoded = X.copy()
for column in X_encoded.select_dtypes(include=['object']).columns:
    X_encoded[column] = LabelEncoder().fit_transform(X_encoded[column])

y_encoded = LabelEncoder().fit_transform(y)

# Calculate Information Gain
mi = mutual_info_classif(X_encoded, y_encoded, discrete_features=True)

# Create a DataFrame to hold feature names and their Information Gain values
feature_info_gain = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Information Gain': mi
})

# Filter features with Information Gain > 0
selected_features = feature_info_gain[feature_info_gain['Information Gain'] > 0]

# Print selected features
print("Features with Information Gain > 0:")
print(selected_features)

# Save the selected features to a new CSV file if needed
selected_features.to_csv('selected_features.csv', index=False)


ValueError: Input X contains NaN.

### Fill Missing Values with a Specific Value

In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

# Load your dataset
data = pd.read_csv('All.csv')

# Identify the target column (assuming it's the last column)
target_column = data.columns[-1]
X = data.iloc[:, :-1]  # All columns except the last one
y = data[target_column]  # Last column as the target

# Handle missing values by filling or dropping
X_filled = X.fillna(X.mean())  # Fill numerical columns with the mean
for column in X_filled.select_dtypes(include=['object']).columns:
    X_filled[column] = X_filled[column].fillna(X_filled[column].mode()[0])

# Handle target variable
y_filled = y.fillna(y.mode()[0])

# Replace infinity values with NaN, then fill them
X_filled.replace([np.inf, -np.inf], np.nan, inplace=True)
X_filled = X_filled.fillna(X_filled.mean())  # Refill any NaN values after replacement

# Convert any remaining large values to a more manageable range if necessary
# For example, you could clip values to a reasonable range
# X_filled = X_filled.clip(-1e10, 1e10)  # Adjust these limits as needed

# Encode categorical features and target variable if necessary
X_encoded = X_filled.copy()
for column in X_encoded.select_dtypes(include=['object']).columns:
    X_encoded[column] = LabelEncoder().fit_transform(X_encoded[column])

y_encoded = LabelEncoder().fit_transform(y_filled)

# Calculate Information Gain
mi = mutual_info_classif(X_encoded, y_encoded, discrete_features=True)

# Create a DataFrame to hold feature names and their Information Gain values
feature_info_gain = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Information Gain': mi
})

# Filter features with Information Gain > 0
selected_features = feature_info_gain[feature_info_gain['Information Gain'] > 0]

# Print selected features
print("Features with Information Gain > 0:")
print(selected_features)

# Save the selected features to a new CSV file if needed
selected_features.to_csv('selected_features.csv', index=False)


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label,

Features with Information Gain > 0:
                  Feature  Information Gain
0             Querylength          0.335475
1      domain_token_count          0.297508
2        path_token_count          0.187484
3       avgdomaintokenlen          0.454635
4      longdomaintokenlen          0.200259
..                    ...               ...
74         Entropy_Domain          1.120500
75  Entropy_DirectoryName          0.860996
76       Entropy_Filename          1.117054
77      Entropy_Extension          0.736039
78      Entropy_Afterpath          0.693892

[78 rows x 2 columns]


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label,

### Drop Rows with Missing Values

In [17]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

# Load your dataset
data = pd.read_csv('All.csv')

# Identify the target column (assuming it's the last column)
target_column = data.columns[-1]
X = data.iloc[:, :-1]  # All columns except the last one
y = data[target_column]  # Last column as the target

# Handle missing values by dropping rows with any NaNs
data_cleaned = data.dropna()

# Separate the cleaned features and target variable
X_cleaned = data_cleaned.iloc[:, :-1]
y_cleaned = data_cleaned[target_column]

# Encode categorical features and target variable if necessary
X_encoded = X_cleaned.copy()
for column in X_encoded.select_dtypes(include=['object']).columns:
    X_encoded[column] = LabelEncoder().fit_transform(X_encoded[column])

y_encoded = LabelEncoder().fit_transform(y_cleaned)

# Calculate Information Gain
mi = mutual_info_classif(X_encoded, y_encoded, discrete_features=True)

# Create a DataFrame to hold feature names and their Information Gain values
feature_info_gain = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Information Gain': mi
})

# Filter features with Information Gain > 0
selected_features = feature_info_gain[feature_info_gain['Information Gain'] > 0]

# Print selected features
print("Features with Information Gain > 0:")
print(selected_features)

# Save the selected features to a new CSV file if needed
selected_features.to_csv('selected_features2.csv', index=False)


Features with Information Gain > 0:
                  Feature  Information Gain
0             Querylength          0.385037
1      domain_token_count          0.291890
2        path_token_count          0.190394
3       avgdomaintokenlen          0.555699
4      longdomaintokenlen          0.273381
..                    ...               ...
74         Entropy_Domain          1.144740
75  Entropy_DirectoryName          1.050474
76       Entropy_Filename          1.061668
77      Entropy_Extension          0.799772
78      Entropy_Afterpath          0.657151

[78 rows x 2 columns]


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label,

In [18]:
all_exploit='All.csv'


In [20]:
all_exploit=pd.read_csv('All.csv')

In [24]:
all_exploit.head()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,...,1,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement
1,0,4,5,5.5,14,6.000000,4,12,4,0,...,0,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement
2,0,4,5,5.5,14,5.800000,4,12,5,0,...,0,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement
3,0,4,12,5.5,14,5.500000,4,32,16,0,...,0,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement
4,0,4,6,5.5,14,7.333334,4,18,11,0,...,0,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement
